In [ ]:
from train_vae import train, test, SpectrogramDataset
from config import load_config
from VAE_models import VAE

from datetime import datetime
import os

from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch
config = load_config()

train_data = SpectrogramDataset(config.data.train_audio_folder)
test_data = SpectrogramDataset(config.data.test_audio_folder)
train_loader = DataLoader(train_data, batch_size=config.vae.batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=config.vae.batch_size, shuffle=False)

sample_spec = train_data[0] # Sample spectogram 
in_dim = sample_spec.numel() # For spectograms = width * height

timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join(config.log_dir, timestamp)
os.makedirs(log_dir, exist_ok=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VAE(in_dim, config.vae.latent_dim, config.vae.n_layers).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=config.vae.lr)
writer = SummaryWriter(log_dir)
prev_updates = 0
for epoch in range(config.vae.epochs):
    print(f'Epoch {epoch+1}/{config.vae.epochs}')
    prev_updates = train(model, train_loader, optimizer, prev_updates, config, device=device, writer=writer)
    test(model, test_loader, prev_updates, config, device, writer=writer)

Epoch 1/50


  0%|          | 0/2260 [00:00<?, ?it/s]

Step 0 (N samples: 0), Loss: 15398.7148 (Recon: 15395.2598, KL: 3.4550) Grad: 447.4013


  0%|          | 4/2260 [00:39<6:13:56,  9.95s/it] 


KeyboardInterrupt: 